### Deploy a local NFS Server

### Deploy a Storage Provider

https://github.com/kubernetes-sigs/nfs-subdir-external-provisioner

In [ ]:
mkdir -p nfs
cd nfs

In [8]:
mkdir -p files

In [9]:
sudo rm -r files/*

rm: das Entfernen von 'files/*' ist nicht möglich: Datei oder Verzeichnis nicht gefunden


: 1

In [10]:
ls files

In [2]:
docker run -d --rm --name nfs \
           --privileged \
           -v $(pwd)/files:/nfsshare \
           -e SHARED_DIRECTORY=/nfsshare \
           --name nfs \
           -p 2049:2049 \
           itsthenetwork/nfs-server-alpine:latest

2e5bd80f3ec6ab0c608c0d9af88ba514e3e9d3ff2cdc0a790834e73796e37bdb


In [12]:
docker exec nfs ip a

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
136: eth0@if137: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default 
    link/ether 02:42:ac:1a:00:02 brd ff:ff:ff:ff:ff:ff link-netnsid 0
    inet 172.26.0.2/16 brd 172.26.255.255 scope global eth0
       valid_lft forever preferred_lft forever


In [4]:
sudo modprobe nfsd

In [19]:
docker ps

CONTAINER ID   IMAGE                                    COMMAND              CREATED         STATUS         PORTS                                       NAMES
b5faf90138bb   itsthenetwork/nfs-server-alpine:latest   "/usr/bin/nfsd.sh"   7 seconds ago   Up 6 seconds   0.0.0.0:2049->2049/tcp, :::2049->2049/tcp   nfs
f55445b90421   openvpn:latest                           "/run.sh"            2 weeks ago     Up 2 weeks                                                 openvpn-denmark


In [20]:
docker exec nfs ip a

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
123: eth0@if124: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default 
    link/ether 02:42:ac:1a:00:03 brd ff:ff:ff:ff:ff:ff link-netnsid 0
    inet 172.26.0.3/16 brd 172.26.255.255 scope global eth0
       valid_lft forever preferred_lft forever


In [21]:
docker logs nfs

Writing SHARED_DIRECTORY to /etc/exports file
The PERMITTED environment variable is unset or null, defaulting to '*'.
This means any client can mount.
The READ_ONLY environment variable is unset or null, defaulting to 'rw'.
Clients have read/write access.
The SYNC environment variable is unset or null, defaulting to 'async' mode.
Writes will not be immediately written to disk.
Displaying /etc/exports contents:
/nfsshare *(rw,fsid=0,async,no_subtree_check,no_auth_nlm,insecure,no_root_squash)

Starting rpcbind...
Displaying rpcbind status...
   program version netid     address                service    owner
    100000    4    tcp6      ::.0.111               -          superuser
    100000    3    tcp6      ::.0.111               -          superuser
    100000    4    udp6      ::.0.111               -          superuser
    100000    3    udp6      ::.0.111               -          superuser
    100000    4    tcp       0.0.0.0.0.111          -          superuser
    100000    3    t

In [8]:
mkdir -p /tmp/docker_nfs

In [13]:
export PROJECT=nfs-provisioner
export NFS_SERVER=172.26.0.2
export NFS_SHARE=/

In [23]:
mkdir -p /tmp/docker_nfs

In [24]:
sudo mount $NFS_SERVER:$NFS_SHARE /tmp/docker_nfs

In [25]:
docker logs nfs

Writing SHARED_DIRECTORY to /etc/exports file
The PERMITTED environment variable is unset or null, defaulting to '*'.
This means any client can mount.
The READ_ONLY environment variable is unset or null, defaulting to 'rw'.
Clients have read/write access.
The SYNC environment variable is unset or null, defaulting to 'async' mode.
Writes will not be immediately written to disk.
Displaying /etc/exports contents:
/nfsshare *(rw,fsid=0,async,no_subtree_check,no_auth_nlm,insecure,no_root_squash)

Starting rpcbind...
Displaying rpcbind status...
   program version netid     address                service    owner
    100000    4    tcp6      ::.0.111               -          superuser
    100000    3    tcp6      ::.0.111               -          superuser
    100000    4    udp6      ::.0.111               -          superuser
    100000    3    udp6      ::.0.111               -          superuser
    100000    4    tcp       0.0.0.0.0.111          -          superuser
    100000    3    t

In [26]:
df

Dateisystem             1K-Blöcke   Benutzt Verfügbar Verw% Eingehängt auf
udev                     32568420         0  32568420    0% /dev
tmpfs                     6525636     21540   6504096    1% /run
/dev/mapper/system-root 102163384  21883848  75020276   23% /
tmpfs                    32628160    473680  32154480    2% /dev/shm
tmpfs                        5120         4      5116    1% /run/lock
tmpfs                    32628160         0  32628160    0% /sys/fs/cgroup
/dev/nvme0n1p1             499656    273620    199824   58% /boot
/dev/mapper/system-var  308587328 263300532  29541772   90% /var
/dev/mapper/system-home 515010816 457667512  31112520   94% /home
/dev/nvme0n1p2             523248      5376    517872    2% /boot/efi
tmpfs                     6525632        32   6525600    1% /run/user/1001
172.26.0.3:/            515011584 457667584  31113216   94% /tmp/docker_nfs


In [13]:
sudo umount /tmp/docker_nfs

## OpenShift Part

In [14]:
oc new-project $PROJECT

Now using project "nfs-provisioner" on server "https://api.crc.testing:6443".

You can add applications to this project with the 'new-app' command. For example, try:

    oc new-app rails-postgresql-example

to build a new example application in Ruby. Or use kubectl to deploy a simple Kubernetes application:

    kubectl create deployment hello-node --image=k8s.gcr.io/e2e-test-images/agnhost:2.33 -- /agnhost serve-hostname



In [15]:
oc adm policy add-scc-to-user privileged system:serviceaccount:$PROJECT:nfs-subdir-external-provisioner

clusterrole.rbac.authorization.k8s.io/system:openshift:scc:privileged added: "nfs-subdir-external-provisioner"


In [16]:
helm repo add nfs-subdir-external-provisioner https://kubernetes-sigs.github.io/nfs-subdir-external-provisioner

Error: repository name (nfs-subdir-external-provisioner) already exists, please specify a different name


: 1

In [17]:
helm upgrade --install nfs-subdir-external-provisioner  \
    nfs-subdir-external-provisioner/nfs-subdir-external-provisioner \
    --set nfs.server=$NFS_SERVER \
    --set nfs.path=$NFS_SHARE

Release "nfs-subdir-external-provisioner" does not exist. Installing it now.
NAME: nfs-subdir-external-provisioner
LAST DEPLOYED: Sat Jun 11 20:28:43 2022
NAMESPACE: nfs-provisioner
STATUS: deployed
REVISION: 1
TEST SUITE: None


In [19]:
oc get all 

NAME                                                   READY   STATUS    RESTARTS   AGE
pod/nfs-subdir-external-provisioner-85b959b6df-f68tp   1/1     Running   0          32s

NAME                                              READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/nfs-subdir-external-provisioner   1/1     1            1           32s

NAME                                                         DESIRED   CURRENT   READY   AGE
replicaset.apps/nfs-subdir-external-provisioner-85b959b6df   1         1         1       32s


In [18]:
oc get events

LAST SEEN   TYPE     REASON              OBJECT                                                    MESSAGE
7s          Normal   LeaderElection      endpoints/cluster.local-nfs-subdir-external-provisioner   nfs-subdir-external-provisioner-85b959b6df-bh4kt_fe52b798-177d-44ad-a9ea-b477609a0bda became leader
10s         Normal   Scheduled           pod/nfs-subdir-external-provisioner-85b959b6df-bh4kt      Successfully assigned nfs-provisioner/nfs-subdir-external-provisioner-85b959b6df-bh4kt to crc-gh9wd-master-0
8s          Normal   AddedInterface      pod/nfs-subdir-external-provisioner-85b959b6df-bh4kt      Add eth0 [10.217.0.150/23] from openshift-sdn
8s          Normal   Pulled              pod/nfs-subdir-external-provisioner-85b959b6df-bh4kt      Container image "k8s.gcr.io/sig-storage/nfs-subdir-external-provisioner:v4.0.2" already present on machine
8s          Normal   Created             pod/nfs-subdir-external-provisioner-85b959b6df-bh4kt      Created container nfs-subdir-externa

In [22]:
helm template nfs-subdir-external-provisioner nfs-subdir-external-provisioner/nfs-subdir-external-provisioner \
    --set nfs.server=172.26.0.2 \
    --set nfs.path=/

---
# Source: nfs-subdir-external-provisioner/templates/serviceaccount.yaml
apiVersion: v1
kind: ServiceAccount
metadata:
  labels:
    chart: nfs-subdir-external-provisioner-4.0.16
    heritage: Helm
    app: nfs-subdir-external-provisioner
    release: nfs-subdir-external-provisioner
  name: nfs-subdir-external-provisioner
---
# Source: nfs-subdir-external-provisioner/templates/storageclass.yaml
apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  labels:
    chart: nfs-subdir-external-provisioner-4.0.16
    heritage: Helm
    app: nfs-subdir-external-provisioner
    release: nfs-subdir-external-provisioner
  name: nfs-client
  annotations:
provisioner: cluster.local/nfs-subdir-external-provisioner
allowVolumeExpansion: true
reclaimPolicy: Delete
parameters:
  archiveOnDelete: "true"
---
# Source: nfs-subdir-external-provisioner/templates/clusterrole.yaml
kind: ClusterRole
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  labels:
    chart: nfs-subdir-external-provisioner-

In [41]:
oc projects | grep nfs

  * nfs-provisioner
Using project "nfs-provisioner" on server "https://api.crc.testing:6443".


In [95]:
oc get sa

NAME                              SECRETS   AGE
builder                           2         37s
default                           2         37s
deployer                          2         37s
nfs-subdir-external-provisioner   2         25s


In [96]:
helm ls

NAME                           	NAMESPACE      	REVISION	UPDATED                                 	STATUS  	CHART                                 	APP VERSION
nfs-subdir-external-provisioner	nfs-provisioner	1       	2022-05-02 13:06:43.994805495 +0200 CEST	deployed	nfs-subdir-external-provisioner-4.0.16	4.0.2      


In [82]:
helm uninstall nfs-subdir-external-provisioner

release "nfs-subdir-external-provisioner" uninstalled


In [97]:
oc get storageclass

NAME         PROVISIONER                                     RECLAIMPOLICY   VOLUMEBINDINGMODE   ALLOWVOLUMEEXPANSION   AGE
nfs-client   cluster.local/nfs-subdir-external-provisioner   Delete          Immediate           true                   39s


In [61]:
oc get storageclass -o yaml

apiVersion: v1
items:
- allowVolumeExpansion: true
  apiVersion: storage.k8s.io/v1
  kind: StorageClass
  metadata:
    annotations:
      meta.helm.sh/release-name: nfs-subdir-external-provisioner
      meta.helm.sh/release-namespace: nfs-provisioner
    creationTimestamp: "2022-05-02T10:49:32Z"
    labels:
      app: nfs-subdir-external-provisioner
      app.kubernetes.io/managed-by: Helm
      chart: nfs-subdir-external-provisioner-4.0.16
      heritage: Helm
      release: nfs-subdir-external-provisioner
    name: nfs-client
    resourceVersion: "51444"
    uid: 9015fc65-4795-4e8d-8a02-2cba66ac6e13
  parameters:
    archiveOnDelete: "true"
  provisioner: cluster.local/nfs-subdir-external-provisioner
  reclaimPolicy: Delete
  volumeBindingMode: Immediate
kind: List
metadata:
  resourceVersion: ""
  selfLink: ""


In [98]:
oc apply -f - <<EOF
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: nfs-device
spec:
  storageClassName: nfs-client
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 1Gi
EOF

persistentvolumeclaim/nfs-device created


In [99]:
oc get pvc | grep nfs-device

nfs-device   Bound    pvc-a8a728ab-0ab5-4cc5-bacb-74ebd80389a0   1Gi        RWO            nfs-client     2s


In [107]:
for i in $(seq 1 10) ;
do
oc apply -f - <<EOF
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: nfs-device-$i
spec:
  storageClassName: nfs-client
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 1Gi
EOF
done

persistentvolumeclaim/nfs-device-1 created
persistentvolumeclaim/nfs-device-2 created
persistentvolumeclaim/nfs-device-3 created
persistentvolumeclaim/nfs-device-4 created
persistentvolumeclaim/nfs-device-5 created
persistentvolumeclaim/nfs-device-6 created
persistentvolumeclaim/nfs-device-7 created
persistentvolumeclaim/nfs-device-8 created
persistentvolumeclaim/nfs-device-9 created
persistentvolumeclaim/nfs-device-10 created


In [108]:
oc get pv,pvc

NAME                                                        CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS      CLAIM                                                 STORAGECLASS   REASON   AGE
persistentvolume/pv0001                                     100Gi      RWO,ROX,RWX    Recycle          Bound       openshift-image-registry/crc-image-registry-storage                           15d
persistentvolume/pv0002                                     100Gi      RWO,ROX,RWX    Recycle          Available                                                                                 15d
persistentvolume/pv0004                                     100Gi      RWO,ROX,RWX    Recycle          Bound       harbor/database-data-my-harbor-database-0                                     15d
persistentvolume/pv0005                                     100Gi      RWO,ROX,RWX    Recycle          Available                                                                                 15d
persistentvolum

In [103]:
oc delete pvc nfs-device

persistentvolumeclaim "nfs-device" deleted


In [109]:
for i in $(seq 1 10) ;
do
oc delete pvc nfs-device-$i
done

persistentvolumeclaim "nfs-device-1" deleted
persistentvolumeclaim "nfs-device-2" deleted
persistentvolumeclaim "nfs-device-3" deleted
persistentvolumeclaim "nfs-device-4" deleted
persistentvolumeclaim "nfs-device-5" deleted
persistentvolumeclaim "nfs-device-6" deleted
persistentvolumeclaim "nfs-device-7" deleted
persistentvolumeclaim "nfs-device-8" deleted
persistentvolumeclaim "nfs-device-9" deleted
persistentvolumeclaim "nfs-device-10" deleted


In [84]:
helm uninstall  nfs-subdir-external-provisioner

Error: uninstall: Release not loaded: nfs-subdir-external-provisioner: release: not found


: 1

In [85]:
helm ls

NAME	NAMESPACE	REVISION	UPDATED	STATUS	CHART	APP VERSION


In [87]:
oc adm policy remove-scc-from-user privileged system:serviceaccount:$PROJECT:nfs-subdir-external-provisioner

Error from server (NotFound): clusterrolebindings.rbac.authorization.k8s.io "system:openshift:scc:privileged" not found


: 1

In [11]:
oc delete project $PROJECT

project.project.openshift.io "nfs-provisioner" deleted


In [19]:
kubectl get storageclass

NAME         PROVISIONER                                     RECLAIMPOLICY   VOLUMEBINDINGMODE   ALLOWVOLUMEEXPANSION   AGE
nfs-client   cluster.local/nfs-subdir-external-provisioner   Delete          Immediate           true                   25m
